In [343]:
import pandas as pd
import numpy as np
import xarray as xr
from scipy.interpolate import interp1d, griddata, NearestNDInterpolator
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
## Loading the historical data.
1. Th#ese are the observations of temperature that we have that date back to 1939 that we want to reconstruct

In [346]:
df = pd.read_excel(r"2. MasterWinterOutputTables_v2.xlsx",
 sheet_name="3-WinterOutput-60rs")
# Concat other datasets here if neccessary
concat_data = df

In [ ]:
# Loading a list of NIWA stations to identify the coordinates
#1. These are not clear or provided in the spreadhse

In [271]:
stations_list = pd.read_table(r"station_list.txt")
stations_list

,Agent,Network,Start_Date,End_Date,Percent_Complete,Name,Lat(dec_deg),Long(dec_deg)
0,6202,L05300,8-Oct-85,5-Mar-20,50,Base Jubany,-62.233,-58.633
1,6207,L06600,26-Feb-99,5-Mar-20,40,Gen San Martin,-68.133,-67.133
2,5952,J56800,1-Jan-81,4-Mar-20,80,"Vanuatu,Aneityum",-20.233,169.767
3,5948,J55500,1-Jan-81,4-Mar-20,80,"Vanuatu,Lamap",-16.417,167.800
4,5946,J55100,1-Jan-81,4-Mar-20,80,"Vanuatu,Sola",-13.850,167.550
...,...,...,...,...,...,...,...,...
3672,4717,H31662,01-Aug-1888,28-Feb-1898,80,"Methven,Drayton",-43.600,171.617
3673,2622,D15133,01-Nov-1883,31-Oct-1888,80,Featherston,-41.117,175.333
3674,3389,E14276,01-Nov-1868,31-Oct-1869,100,"Wellington, Bowen St",-41.283,174.783
3675,3383,E14270,01-Feb-1862,31-Oct-1868,100,"Wellington, Knowles Observatory",-41.283,174.783


In [223]:
# Reference times to load the data for
times = pd.date_range("1939-01-01", periods=500, freq ='1D')
dset_temp = pd.DataFrame(index = times)
dset_pressure = pd.DataFrame(index = times)
# Looping through the networks and extracting data
networks = concat_data['File name'].unique()
# Creatin temporary dataframes to store the data

In [295]:

site_metadata = {}
for site in networks:
    # Looping through
    subset = concat_data[concat_data['File name'] == site]
    subset.index = pd.to_datetime(subset['Date'])
    # subsetting the data from here
    # get the coordsinates
    # Finding the minmim temperature
    temp = subset['Min temp (oF)']
    temp = temp[(temp> -999) & (temp <100)]
    # Filtering values outside of the expected bounds

    pressure = subset['Barometer corrected']
    pressure = pressure[(pressure > -999) & (pressure < 1040)]
    # Applying a similar check for pressure
    pressure[(pressure>25.4)& (pressure< 31.0)] = pressure[(pressure>25.4)& (pressure< 31.0)] * 33.86
    # Discharding invalid values outside of the expected range - and converting to hPa
    pressure = pressure[pressure > 960]
    if (pressure.mean() < 998)|(len(pressure.dropna())< 30):
        pressure =[]
    # Tests for average pressure
    agent = stations_list[stations_list.Network == site].Agent
    print(stations_list[stations_list.Network == site])
    if len(agent) >0:
        site_metadata[site] ={}
        site_metadata[site]['lat'] = stations_list[stations_list.Network == site]['Lat(dec_deg)'].values[0]
        site_metadata[site]['lon'] = stations_list[stations_list.Network == site]['Long(dec_deg)'].values[0]
        # Saving the metadata also
        if len(temp)>0:
            #print(temp)
            dset_temp[site] = temp
        if len(pressure)>0:
            dset_pressure[site] = pressure


      Agent Network   Start_Date   End_Date  Percent_Complete  \
1436   1427  A64871  01-Jan-1863  28-Feb-94               100   

                       Name  Lat(dec_deg)  Long(dec_deg)  
1436  Auckland, Albert Park       -36.853        174.767  
      Agent Network Start_Date   End_Date  Percent_Complete  \
2942   1005  A42581   1-Jul-31  30-Sep-73                90   

                       Name  Lat(dec_deg)  Long(dec_deg)  
2942  Te Paki Stn, Te Hapua       -34.508        172.795  
      Agent Network Start_Date   End_Date  Percent_Complete        Name  \
2926   1085  A53291  13-Feb-45  31-Dec-73               100  Kerikeri 1   

      Lat(dec_deg)  Long(dec_deg)  
2926        -35.23        173.948  
      Agent Network Start_Date   End_Date  Percent_Complete              Name  \
2210   1405  A64751   1-Apr-28  31-Mar-87               100  Riverhead Forest   

      Lat(dec_deg)  Long(dec_deg)  
2210       -36.766        174.579  
Empty DataFrame
Columns: [Agent, Network, Start_

In [255]:
datasets = pd.DataFrame(site_metadata)
datasets.to_csv('Site_coordsinates.csv')
# Determining sites o finterest

In [ ]:
sites = (dset_temp.isna().sum() < 480) #& (dset_pressure.isna().sum() < 350)
temps = dset_temp[sites[sites == True].index]
#pressure = dset_pressure[sites[sites == True].index]


# Regridding the site data using nearest neightbour interpolation

In [338]:
sites = pd.read_csv('Site_coordsinates.csv', index_col=0)
# Loading the sites
# using the sites that are within the north island
sites = sites[['A64871', 'A42581', 'A53291', 'A64751', 'B75181', 'B75381', 'B75571',
       'B76621', 'B76951', 'B86124', 'C75731', 'C94001', 'C95251', 'D05481',
       'D05960', 'D87692', 'D87811', 'D96591', 'D96681', 'E04891', 'E05363',
       'E14272', 'E95451', 'E95681', 'E95902', 'F20791', 'F38961', 'F47691',
       'G12092', 'G12581', 'G13211', 'G13231', 'G13592', 'G13651', 'G22581',
       'H22862', 'H31352', 'H31562', 'H31971', 'H32412', 'H32552', 'H32561',
       'H32641', 'H32791', 'H40041', 'H40182', 'H41421', 'H41701', 'I50212',
       'I50852', 'I58061', 'I59161', 'I59234', 'I59361', 'I68093']]
sites = sites.T[(sites.T.lat >-40.7) & (sites.T.lon> 173.5)].T
temps[sites.columns].dropna()

,A64871,A53291,A64751,B75181,B75381,B75571,B76621,B76951,B86124,C75731,...,C95251,D05481,D87692,D87811,D96591,D96681,E05363,E95451,E95681,E95902
1939-06-01,45.0,45.8,30.0,39.0,34.0,40.5,40.0,35.7,34.4,30.2,...,33.2,37.0,43.3,39.9,41.8,39.0,40.0,31.1,40.0,42.0
1939-06-02,47.6,43.4,32.0,35.8,31.5,45.0,40.5,31.3,32.7,30.8,...,26.8,29.0,41.5,37.8,39.5,35.4,32.6,24.1,34.0,38.2
1939-06-03,44.0,39.5,29.7,46.3,38.5,36.0,41.8,36.1,32.0,27.6,...,28.6,37.6,38.3,38.8,36.5,32.0,42.0,25.8,34.2,42.0
1939-06-04,49.4,48.0,37.5,37.0,36.0,35.0,39.5,32.2,30.8,30.0,...,33.6,33.4,37.4,38.5,40.3,37.2,40.0,24.5,35.6,42.8
1939-06-06,57.3,57.2,48.5,51.5,46.0,48.0,52.0,51.0,47.8,49.6,...,33.4,53.0,52.7,49.5,46.8,47.0,46.2,47.5,49.2,54.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1939-08-23,51.2,52.0,43.0,38.8,40.0,43.0,45.5,41.5,40.3,45.1,...,31.4,37.0,36.2,39.5,41.0,38.6,43.9,30.5,40.0,53.0
1939-08-24,48.0,42.7,35.0,37.0,43.5,46.0,43.0,37.4,40.9,40.0,...,32.7,45.0,42.5,44.0,46.0,37.6,47.2,39.1,40.2,43.5
1939-08-25,50.8,47.2,43.0,50.0,51.0,43.0,50.5,49.2,45.1,50.2,...,32.8,37.6,46.4,41.2,46.0,44.2,49.5,37.9,45.0,47.0
1939-08-27,47.5,43.9,41.2,49.0,44.5,45.5,43.5,40.9,38.0,42.4,...,34.2,44.0,41.0,42.2,45.4,41.4,50.0,38.5,41.0,46.6


In [339]:
# creating an xarray dataset, and gridding using a unfguc
d ={}
d['time'] = (('time'), temps[sites.columns].dropna().index)
d['z'] = (('z'), np.arange(len(sites.columns)))
d['data'] =(('time','z'), temps[sites.columns].dropna().values)
dset = xr.Dataset(d)


## Finding the nearest coordinates in the 2d dataset to our sites

In [340]:
df2 = xr.open_dataset(r"C:\Users\rampaln\OneDrive - NIWA\Desktop\week-it-snowed-everywhere\data\highresolution_northland_temperature_data.nc")
df21 = df2.reindex(time1 = df2.time1.to_index().sort_values())

sites = pd.read_csv('Site_coordsinates.csv', index_col=0)
sites = sites[['A64871', 'A42581', 'A53291', 'A64751', 'B75181', 'B75381', 'B75571',
       'B76621', 'B76951', 'B86124', 'C75731', 'C94001', 'C95251', 'D05481',
       'D05960', 'D87692', 'D87811', 'D96591', 'D96681', 'E04891', 'E05363',
       'E14272', 'E95451', 'E95681', 'E95902', 'F20791', 'F38961', 'F47691',
       'G12092', 'G12581', 'G13211', 'G13231', 'G13592', 'G13651', 'G22581',
       'H22862', 'H31352', 'H31562', 'H31971', 'H32412', 'H32552', 'H32561',
       'H32641', 'H32791', 'H40041', 'H40182', 'H41421', 'H41701', 'I50212',
       'I50852', 'I58061', 'I59161', 'I59234', 'I59361', 'I68093']]
sites = sites.T[(sites.T.lat >-40.7) & (sites.T.lon> 173.5)].T
coords = sites.values.T
# Discarding too far south


# writing a function to find the nearest coordinates
def min_nearest_coords(lat, lon):
    dist = ((lats - lat)**2 + (lons - lon)**2)
    return np.where(dist == dist.min())


df22 = df21.isel(rlat = slice(25,416+25), rlon = slice(30,256 +30))
subset = df22
lats, lons = df22.latitude.values, df22.longitude.values
coords_in_arr = np.array([min_nearest_coords(*coord) for coord in coords])[:,:,0]
# finds the correct coordinates in the array

In [337]:
coords_in_arr

array([[275,  73],
       [397,  28],
       [282,  62],
       [249, 137],
       [232, 135],
       [221, 127],
       [211, 155],
       [193, 172],
       [174, 157],
       [205, 103],
       [112,  28],
       [ 99, 112],
       [  6, 123],
       [132, 255],
       [124, 204],
       [ 74, 189],
       [ 64, 185],
       [ 12, 112],
       [ 79, 109],
       [ 65, 127],
       [ 46,  81]], dtype=int64)

# Interpolating using nearest neightbour interpolation in parallel

In [341]:
def merge_data(x, pts =[416,256]):
    print("merging")
    xi, yi = np.meshgrid(np.arange(pts[1]), np.arange(pts[0]))
    interp = NearestNDInterpolator(list(coords_in_arr), x.astype('int8'))
    return interp(xi, yi).astype('int8')
# Using nearest neigbour interpolation to initialize the outputs


train_set = xr.apply_ufunc(merge_data, dset,
                           input_core_dims=[["z"]],
                           output_core_dims=[["x","y"]],
                           vectorize=True, dask='parallelized')

merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging


In [353]:
train_set['data'].to_netcdf('the_week_it_snowed_gridded.nc')